In [1]:
%load_ext autoreload
%autoreload 2
%cd ..

/home/ubuntu/latent-diffusion-lm


In [2]:
from src.task.optimus_v2 import OptimusTask
from src.task.sgpt import SGPTTask
from coop.metric import levenshtein_batch, levenshtein
import torch
from datasets import load_dataset

/home/ubuntu/.local/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# sgpt = SGPTTask.load_from_checkpoint("outputs/2022-10-18/06-46-26/checkpoint/sgpt-tiny-opt44M.ckpt") # first
# sgpt = SGPTTask.load_from_checkpoint("outputs/2022-10-20/00-22-07/checkpoint/sgpt-base-opt44M.ckpt") # mse 학습
sgpt = SGPTTask.load_from_checkpoint("outputs/2022-10-20/03-20-29/checkpoint/sgpt-base-opt56M.ckpt").eval().cuda() # kl-div로 학습

In [5]:
torch.set_grad_enabled(False)


자주 가는 맛집 있으세요?
동네에 돌곱창집 진짜 자주갓는데ㅜ
영업시간제한있고는 못갔어요ㅠ
아 곱창 맛있는데요!!
양념곱창볶음인데ㅜㅜ 진짜 넘 맛있어요 ㅎㅎㅎ

In [10]:
import torch.nn.functional as F

def get_next_sentence(context, answer):
    latents = sgpt.autoencoder.encode(context.strip().split("\n"))
    answer_latent = sgpt.autoencoder.encode(answer)
    response_latent = sgpt.model(inputs_embeds=latents, attention_mask=None).latent.loc[-1, :].unsqueeze(0)
    # print(response.shape)
    response = sgpt.autoencoder.generate(response_latent, max_length=64, num_beams=4, min_length=10)[0]
    # mse = ((response_latent - answer_latent) ** 2).sum(-1)
    mse = F.mse_loss(response_latent, answer_latent)
    print(response, mse.item())
    

get_next_sentence("""
자주 가는 맛집 있으세요?
동네에 돌곱창집 진짜 자주갓는데ㅜ
""",
"동네에 돌곱창집 진짜 자주갓는데ㅜ")

저기의 선물도 아니엿해도 항상 2 0.07205158472061157


In [6]:
# sgpt.autoencoder.generate(sgpt.autoencoder.encode("저 여럿안하는 모아가 없네"))
# sgpt.autoencoder.generate(sgpt.autoencoder.encode("동네에 돌곱창집 진짜 자주갓는데ㅜ"))

# F.kl_div(
#     torch.FloatTensor([[1, 2, 3, 0]]),
#     torch.FloatTensor([[1, 2, 3, 0]]),
#     reduction='none',
#     log_target=True
# )
sgpt.autoencoder.encode("동네에 돌곱창집 진짜 자주갓는데ㅜ", return_distribution=True)

Normal(loc: torch.Size([1, 512]), scale: torch.Size([1, 512]))

In [15]:
import torch
a1 = torch.FloatTensor([[1.4, 0.5, 0.1]])
a2 = torch.FloatTensor([[2.0, 0.6, 0.1]])
c1 = torch.nn.KLDivLoss(reduction='none')(a1.log(),a2)
print(c1)

tensor([[0.7133, 0.1094, 0.0000]])


In [13]:
from typing import Optional
from torch.distributions import Normal, kl_divergence

def kldiv_loss(p: Normal, attention_mask: Optional[torch.Tensor] = None):
    zkl_real = kl_divergence(p, Normal(0, 1))

    if attention_mask is not None:
        attn_sum = attention_mask.sum()
        zkl_real = zkl_real.masked_fill(attention_mask == 0, 0.0)
    else:
        attn_sum = p.loc.shape[0] * p.loc.shape[1]

    kl_mask = torch.gt(zkl_real, 0.5)
    zkl = zkl_real[kl_mask].sum() / attn_sum
    zkl_real = zkl_real.sum() / attn_sum

    return zkl, zkl_real

kldiv_loss(
    Normal(a1, a2),
    torch.LongTensor([[1, 1, 0]])
)

(tensor(0.8934), tensor(1.0513))

In [34]:
n1 = Normal(a1.squeeze(0), a2.squeeze(0))
n2 = Normal(a2.squeeze(0), a1.squeeze(0))
n1, n2

def stack_normals(normals):
    locs = [x.loc for x in normals]
    scales = [x.scale for x in normals]
    return Normal(
        torch.stack(locs),
        torch.stack(scales),
    )

stack_normals([n1, n2]).loc

tensor([[1.4000, 0.5000, 0.1000],
        [2.0000, 0.6000, 0.1000]])